In [1]:
import pandas as pd
import requests
import json
from pprint import pprint
import time
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder

# Data Import
Calling API and Importing JSON File

In [2]:
url_current = "https://services5.arcgis.com/54falWtcpty3V47Z/arcgis/rest/services/general_offenses_year3/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&resultType=standard&f=json"
url_one_year_ago = 'https://services5.arcgis.com/54falWtcpty3V47Z/arcgis/rest/services/general_offenses_year2/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
url_two_year_ago = 'https://services5.arcgis.com/54falWtcpty3V47Z/arcgis/rest/services/general_offenses_year1/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

json_current = json.loads(requests.get(url_current).text)
json_one_year_ago = json.loads(requests.get(url_one_year_ago).text)
json_two_year_ago = json.loads(requests.get(url_two_year_ago).text)

# Reformating Data
* Pulling values from each key and listing them in a seperate variable
* Creating a dictionary with our new lists
* Transforming Dictionary to DataFrame

In [3]:
def json_converter(json):
    
    Beat = []
    Description = []
    Grid = []
    ObjectID = []
    Occurence_Date = []
    Offense_Category = []
    Offense_Code = []
    Offense_Ext = []
    Police_District = []
    Record_ID = []

    for i in range(len(json['features'])):
        for k, v in json['features'][i]['attributes'].items():
            if k == 'Beat':
                Beat.append(v)
            if k == 'Description':
                Description.append(v)
            if k == 'Grid':
                Grid.append(v)
            if k == 'OBJECTID':
                ObjectID.append(v)
            if k == 'Occurence_Date':
                Occurence_Date.append(v)
            if k == 'Offense_Category':
                Offense_Category.append(v)
            if k == 'Offense_Code':
                Offense_Code.append(v)
            if k == 'Offense_Ext':
                Offense_Ext.append(v)
            if k == 'Police_District':
                Police_District.append(v)
            if k == 'Record_ID':
                Record_ID.append(v)
                
    crime_dict = {'Beat': Beat,
                  'Description': Description,
                  'Grid': Grid,
                  'ObjectID': ObjectID,
                  'Occurence_Date': Occurence_Date,
                  'Offense_Category': Offense_Category,
                  'Offense_Code': Offense_Code,
                  'Offense_Ext': Offense_Ext,
                  'Police_District': Police_District,
                  'Record_ID': Record_ID}
    
    crime_df = pd.DataFrame(crime_dict).set_index('ObjectID')
    
    return crime_df

In [4]:
def crime_cleanup(crime_df):
    
    important_crimes = ['AUTO THEFT','BURGLARY','GRAND THEFT','LARCENY','PRIVACY','ROBBERY','STOLEN PROP','THEFT','TREPASS','TRESPASS']
    crime_df = crime_df.loc[crime_df['Beat'] != 'UI']
    crime_df = crime_df.loc[crime_df['Offense_Category'].isin(important_crimes)]
    crime_df.dropna(inplace=True)
    crime_df.reset_index(drop=True, inplace=True)
    
    return crime_df

# Converting Data Types

In [5]:
def data_cleanup(crime_df):
    
    crime_df['Grid'] = crime_df['Grid'].astype('int')
    crime_df['Occurence_Date'] = crime_df['Occurence_Date'].astype('str')
    crime_df['Offense_Code'] = crime_df['Offense_Code'].astype('int')
    crime_df['Police_District'] = crime_df['Police_District'].astype('int')
    crime_df['Record_ID'] = crime_df['Record_ID'].astype('int')
    
    # Converting unix code to readble dates and times
    Occurence_Date = []
    Day_of_the_Week = []
    Month = []
    Day_of_the_Month = []
    Year = []
    Hour = []
    Minute = []
    
    for i in range(len(crime_df)):
        trimmed_unix = crime_df['Occurence_Date'][i][0:10]
        day_of_week = time.strftime('%u', time.localtime(int(trimmed_unix)))
        month = time.strftime('%m', time.localtime(int(trimmed_unix)))
        day_month = time.strftime('%d', time.localtime(int(trimmed_unix)))
        year = time.strftime('%Y', time.localtime(int(trimmed_unix)))
        hour = time.strftime('%H', time.localtime(int(trimmed_unix)))
        minute = time.strftime('%M', time.localtime(int(trimmed_unix)))
        date_time = time.strftime('%a %m-%d-%Y, %I:%M:%S %p', time.localtime(int(trimmed_unix)))
        
        date_time_object = datetime.strptime(date_time, '%a %m-%d-%Y, %I:%M:%S %p')
        rounded_date_time_object = date_time_object + (datetime.min - date_time_object) % timedelta(minutes=30)
        rounded_date_time = rounded_date_time_object.strftime('%a %m-%d-%Y, %I:%M:%S %p')
        
        Occurence_Date.append(rounded_date_time)
        Day_of_the_Week.append(day_of_week)
        Month.append(day_month)
        Day_of_the_Month.append(day_month)
        Year.append(year)
        Hour.append(hour)
        Minute.append(minute)

    crime_df['Occurence_Date'] = Occurence_Date
    crime_df['Day_of_the_Week'] = Day_of_the_Week
    crime_df['Month'] = Month
    crime_df['Day_of_the_Month'] = Day_of_the_Month
    crime_df['Year'] = Year
    crime_df['Hour'] = Hour
    crime_df['Minute'] = Minute
    
    # Encoding beats to numerical values
    label_encoder = LabelEncoder()
    label_encoder.fit(crime_df['Beat'])
    encoded_beat = label_encoder.transform(crime_df['Beat'])
    
    crime_df['Encoded_Beat'] = encoded_beat
    
    return crime_df

In [6]:
crime_current_df = data_cleanup(crime_cleanup(json_converter(json_current)))
crime_one_year_ago_df = data_cleanup(crime_cleanup(json_converter(json_one_year_ago)))
crime_two_year_ago_df = data_cleanup(crime_cleanup(json_converter(json_two_year_ago)))

In [7]:
crime_df = crime_two_year_ago_df.append(crime_one_year_ago_df).append(crime_current_df).reset_index(drop=True)

In [8]:
crime_df

,Beat,Description,Grid,Occurence_Date,Offense_Category,Offense_Code,Offense_Ext,Police_District,Record_ID,Day_of_the_Week,Month,Day_of_the_Month,Year,Hour,Minute,Encoded_Beat
0,3M,602(L)(1)TRESPASS REFUSE TO LV,734,"Mon 01-01-2018, 02:30:00 AM",TRESPASS,5707,6,3,1287465,1,01,01,2018,02,15,8
1,5B,459 PC BURG RESIDENCE-FORCE,1604,"Mon 01-01-2018, 05:00:00 PM",BURGLARY,2202,0,5,1287470,1,01,01,2018,16,45,13
2,1C,459 PC BURG BUSINESS-FORCE,309,"Mon 01-01-2018, 06:00:00 AM",BURGLARY,2203,0,1,1287505,1,01,01,2018,05,54,2
3,2A,459 PC BURG RESIDENCE-FORCE,222,"Mon 01-01-2018, 03:30:00 AM",BURGLARY,2202,0,2,1287511,1,01,01,2018,03,17,3
4,6C,459 PC BURG RESIDENCE-FORCE,1445,"Mon 01-01-2018, 08:00:00 AM",BURGLARY,2202,0,6,1287564,1,01,01,2018,08,00,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7056,1C,484 PETTY THEFT CLOUT UNLOCKED,405,"Sun 06-07-2020, 01:30:00 PM",LARCENY,2305,0,1,1460434,7,07,07,2020,13,30,2
7057,6C,484 PC PETTY THEFT,1111,"Thu 05-21-2020, 07:00:00 PM",LARCENY,2399,2,6,1460436,4,21,21,2020,18,44,17
7058,3A,484 PC PETTY THEFT,785,"Tue 06-09-2020, 10:00:00 AM",LARCENY,2399,2,3,1460506,2,09,09,2020,09,37,6
7059,1A,484 PETTY THEFT/LICENSE PLATE,348,"Wed 05-20-2020, 03:00:00 PM",LARCENY,2399,4,1,1460861,3,20,20,2020,15,00,0


# Placing Data Frame into a CSV

In [9]:
crime_df.to_csv('sac_crime.csv', index=False)